## Nextgenmap
Runs `nextgenmap` in paired data setting. Samples are grouped by R1 and R2 files. 


In [1]:
import logging
import os
import subprocess
import sys

from hops import hdfs
from pyspark import SparkContext

import utils


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
319,application_1613482861329_0025,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
args=utils.load_arguments(sys.argv)


if args is not None:
    args=args['Nextgenmap']
else :
    sys.exit(utils.NO_CONFIG_ERR)
    
sc = SparkContext.getOrCreate()

Started copying hdfs:///Projects/4C/Jupyter/settingsDJ.yml to local disk on path /srv/hops/hopsdata/tmp/nm-local-dir/usercache/PmPpORgvdLjbK5ycqQzUncHEjlUNaHsCD0bOZMp-rKA/appcache/application_1613482861329_0025/container_e40_1613482861329_0025_01_000001/

Finished copying

In [5]:
VERY_FAST='--very-fast'
INPUT_ROOT=args['INPUT_ROOT']
OUTPUT_ROOT=args['OUTPUT_ROOT']
REFERENCE_PATH=args['REFERENCE_FILE']
MIN_I=str(args['MIN-IDENTITY'])
MIN_R=str(args['MIN-RESIDUES'])
THREADS=args['THREADS']
LOG_DIR=args['LOGS_ROOT']
is_very_fast=args['VERY_FAST']
SPACE=utils.SPACE
REFERENCE_FILE=os.path.split(REFERENCE_PATH) [1]

In [ ]:
    

pairedList=utils.load_file_names(INPUT_ROOT)


In [ ]:
# group r1 and r2
pairedList = utils.group_R1R2(pairedList)

In [ ]:

def apply_ngm(x,REFERENCE_PATH): ### paired files
    
    
    filename_forward=x[0] #r1
    filename_reverse=x[1] #r2
     
    hdfs.copy_to_local(REFERENCE_PATH)
    hdfs.copy_to_local(os.path.join(INPUT_ROOT,filename_forward), overwrite=True)
    hdfs.copy_to_local(os.path.join(INPUT_ROOT,filename_reverse), overwrite=True)
             
    output_file=filename_forward.split('.')[0].replace('R1_','')+'.sam'
    
    parameters = { '-i':MIN_I, '-R': MIN_R, '-p': utils.EMPTY, '-r': REFERENCE_FILE, '-1': filename_forward, '-2': filename_reverse,
                  '--silent-clip': utils.EMPTY, '-o': output_file, '-t': THREADS, '--no-progress': utils.EMPTY }
    
    cmd = utils.build_command('ngm',parameters)
    
    if is_very_fast :
        cmd=cmd+SPACE+VERY_FAST
    
    logging.info('Running nextgenmap with command:', cmd)
    log_file=os.path.splitext(output_file) [0]+'.txt'
    f=open(log_file, "w")
    
    subprocess.run(cmd.split(' '),stdout=f,stderr=f)
    
    hdfs.copy_to_hdfs(log_file, LOG_DIR, overwrite=True)
    f.close()
    status=False
    if os.path.exists(output_file):
        
        hdfs.copy_to_hdfs(output_file, OUTPUT_ROOT, overwrite=True)
        status=True
        os.remove(filename_forward)
        os.remove(filename_reverse)
        os.remove(output_file)
        os.remove(log_file)
        
    parameters.clear()

    return [status,output_file]
            

In [ ]:
dataPairedRdd=sc.parallelize(pairedList)


In [ ]:
# run 
ngmFiles=dataPairedRdd.map(lambda x: apply_ngm(x,REFERENCE_PATH) )